# Probabilistic Graphical Models

The roots of probabilistic graphical models go back to the 1980s (see the works of Judea Pearl), with a strong connection to Bayesian statistics (see Bayes, T. and Price, R. (1763). An Essay towards solving a Problem in the Doctrine of Chances”. *Phil. Trans.*, 53, 370–418. doi:[10.1098/rstl.1763.0053](https://doi.org/10.1098/rstl.1763.0053)). The story resembles that of neural networks:

-   They have been around for over three decades.

-   They were enabled by computational advances.

Now we have

-   Efficient computational frameworks for probabilistic inference.

-   Quantum resources.

Probabilistic graphical models capture a compact representation of a joint probability distribution. For $\{X_1,\ldots,X_N\}$ binary random variables, there are $2^N$ assignments. In a graphical model, complexity is dealt through graph theory. We get both an efficient treatment of uncertainty (probabilities) and of logical structure (independence constraints). The factorization of the probabilities happens along conditional independences among random variables. The definition is $(X\perp Y|Z)$:

$$P(X=x, Y=y|Z=z) = P(X=x|Z=z)P(Y=y|Z=z)$$

$$\forall x\in X,y\in Y,z\in Z$$.

The graph can be directed -- these are called Bayesian networks in general -- or undirected, in the case of Markov networks (also known as Markov random fields). For a Bayesian network, the underlying graph is a directed acyclic graph (DAG), where nodes are random variables and edges indicate direct influence. We have a local normalization at each node: a conditional probability distribution of the form $P(X_i|\mathbf{Pa}_{X_i})$, where $\mathbf{Pa}_{X_i}$ indicates the parent nodes of a random variable $X_i$ (see Koller et al. (2007). Graphical Models in a Nutshell for more details). Graphical models are quintessentially generative: we explicitly model a probability distribution. Thus generating new samples is trivial and we can always introduce extra random variables to ensure certain properties. These models also take us a step closer to explainability, either by the use of the random variables directly for explanations (if your model is such) or by introducing explanatory random variables that correlate with the others.

A core result in Bayesian networks is the d-separation theorem that tells you the exact graph structures that are permitted for a factorization of the form

$P(X_1,\ldots,X_N)=\prod_{i=1}^N P(X_i|\mathbf{Pa}_{X_i})$.

In turn, we are looking at conditional independences of the form $(X_i\perp \textrm{NonDescendants}_{X_i}|\mathbf{Pa}_{X_i})$. Note that the causal structure is **not** captured entirely by the directed graph part of the statistical model.

In a Markov random field, we can allow cycles in the graph and switch from local normalization (conditional probability distribution at each node) to global normalization of probabilities (i.e. a partition function). Examples include countless applications in computer vision, pattern recognition, artificial intelligence, but also Ising models that we have seen before: the factors are defined as degree-1 and degree-2 monomials of the random variables connected in the graph.

The factorization is given as a sum:

$P(X_1, \ldots, X_N) = \frac{1}{Z}\exp(-\sum_i E[C_k]),$

where $C_k$ are are cliques of the graph, and $E[.]$ is an energy defined over the cliques.

The local Markov property says that a state is independent from the rest of the network given its immediate neighbours:

$I_l(G) = \{(X\perp \mathcal{X}-\{X\}-N_G(X)|N_G(X)): X\in\mathcal{X}\}$

If $P$ is a Gibbs distribution over $G$, all local Markov properties will hold. The other way also holds if $P$ is a positive distribution.

The need to sample. #P-complete

It is highly non-trivial to learn the structure of a PGM given a set of observed correlations in the sample $S$. We can only rely on heuristics. The typical way of doing it is to define a scoring function and do some heuristic global optimization like simulated annealing. The alternative is to define the graph structure, as in the case of the nearest-neighbour Ising model.

Once we identified or defined the graph structure $G$, we have to learn the probabilities in the graph. We again rely on our sample and its correlations, and use an MLE or a MAP estimate of the corresponding parameters $\theta_G$ with the likelihood $P(S|\theta_G)$. This is again a hard problem.

In every model we have seen so far, learning is hard, but applying the learned model is computationally inexpensive. This is not the case in generic PGMs. Applying the learned model means probabilistic inference to answer queries of the following types:

-   Conditional probability: $P(Y|E=e)=\frac{P(Y,e)}{P(e)}$.

-   Maximum a posteriori:
    $\mathrm{argmax}_y P(y|e)=\mathrm{argmax}_y \sum_Z P(y, Z|e)$.

This is NP-hard to do exactly, in fact, it is in \#P. This is why we do approximate inference instead. In physics, we commonly use the mean field method, where we assume that the posterior fully factorizes as $q(\mathbf{x}) = \prod_{i=1}^{N} q_i(\mathbf{x}_i)$. Then we optimize over each marginal distribution $q_i$. The update is given by

$\log q_j(\mathbf{x}_j) = \mathbb{E}_{-q_j}\left[\log p(\mathbf{X}, D)\right] + \mathrm{constant}$,

where $-q_j$ means all variables except $q_j$. In other words, we replace the neighbors with their average values.

The second option for approximate inference is sampling, which is primarily done with Markov chain Monte Carlo Gibbs sampling. We aim at constructing a Markov chain with stationary distribution matching the target, essentially doing a random walk on the state space. The time spent in each state $\mathbf{x}$ is proportional to the distribution. The problem is that in the Markov chain, the samples are correlated. Furthermore, the Markov chain has a burn-in phase: the mixing time defines how long it takes to reach the stationary distribution. In general, it is difficult to tell when we reached the target distribution.

In Gibbs sampling, we perturb random variables one after the other. For instance, in an Ising model, we flip one spin at a time. For approximating the thermal state, accept a new configuration     depending on energy decrease and the temperature. If we add cooling to Gibbs sampling, we end up with simulated annealing.

# Training a Markov Network